In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [22]:
data = pd.read_csv('earnings_data.csv')
data = pd.get_dummies(data, drop_first=True)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('salary', axis = 1),
                                                    np.log1p(data.salary.values),
                                                    test_size =.30,
                                                    random_state=12345)
dtrain = xgb.DMatrix(data = X_train, label=y_train)
dtest = xgb.DMatrix(data = X_test, label=y_test)
param = {'eta':0.1,
         'objective':'reg:linear'}
xgb_model = xgb.train(param, dtrain, num_boost_round=100)
X_test['y_hat_with_bias'] = np.exp(xgb_model.predict(dtest)) - 1

[19:08:50] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


In [24]:
feature_names = dtest.feature_names
interactions = xgb_model.predict(dtest, pred_interactions=True)
pd.DataFrame(interactions[0],
             index = feature_names + ['intercept'],
             columns= feature_names + ['intercept'])

,years_of_experience,gender_male,career_se,intercept
years_of_experience,0.249059,-0.012967,0.011549,0.000000
gender_male,-0.012967,-0.059626,0.006620,0.000000
career_se,0.011549,0.006620,-0.005303,0.000000
intercept,0.000000,0.000000,0.000000,11.336657


In [25]:
bias_var = np.array('gender_male')
bias_idx = np.argwhere(np.isin(np.array(feature_names), bias_var))[0]
interactions[:, bias_idx, :] = 0
interactions[:, :, bias_idx] = 0
y_hat_no_bias = np.exp(interactions.sum(axis = 1).sum(axis = 1)) - 1

In [26]:
print(y_hat_no_bias)

[109480.51   99489.766 105018.375  72244.15   99822.79   84569.734
  70040.836  72639.11   99489.766 106268.67   70481.74   79874.71
  84963.516 106923.36   96219.04   67091.125  63111.21   99822.79
  96219.04   86454.81   94811.61   78616.03   92830.24   92855.92
 102644.62  105018.375  58180.02  105564.63  105872.64  102078.22
 104857.75   63574.242  67074.625  63210.6    78828.27  103839.414
  96118.24   72244.15  105409.61   62748.     98533.625 102078.22
  80745.15   62748.     93918.15   72244.15   72639.11  103839.414
  96118.24  103697.805  78828.27   62672.168  72337.984  70481.74
  58423.605  58423.605  70680.445  88788.695  78828.27   62748.
 106268.67   58180.02   92855.92   80768.875  95290.32   88788.695
  80745.38   62748.    105018.375  58423.605  97213.9    99558.58
  97213.9    98533.625  72858.56  102078.22   80067.74   93331.88
 109480.51   72639.11   80745.38   93918.15   72858.56  104857.75
  70229.79   72742.48   99489.766  72742.48   80745.15   97213.9
  84569.7

In [11]:
plt.scatter(X_test_male, y_test_male,  color='black')
plt.scatter(X_test_female, y_test_female,  color='red')
plt.plot(X_test_male, y_pred_male, color='blue', linewidth=3)
plt.plot(X_test_female, y_pred_female, color='green', linewidth=3)

#plt.xticks(())
#plt.yticks(())

plt.show()

NameError: name 'plt' is not defined

In [12]:
print(residuals_plot(xgb_model, X_train, y_train, X_test, y_test))

NameError: name 'residuals_plot' is not defined

In [ ]:
from yellowbrick.regressor import residuals_plot
from yellowbrick.regressor import prediction_error